<a href="https://colab.research.google.com/github/tmyc131/lrc_file_crawler/blob/master/kanana_1_5_2_1b_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

Model page: https://huggingface.co/kakaocorp/kanana-1.5-2.1b-base

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/kakaocorp/kanana-1.5-2.1b-base)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="kakaocorp/kanana-1.5-2.1b-base")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'When was your first video made?\xa0\n\nWhen did you start your YouTube Channel?\xa0\n\nHave you got any advice for aspiring YouTubers?\xa0\n\nIn your opinion, where is YouTube heading?\xa0\n\nWhat are you working on now?\xa0\n\nCan you tell us about the future of YouTube?\xa0\n\nWhat are people saying about you?\xa0\n\nWho are some key YouTubers who inspire you?\xa0\n\nWhat is your favourite YouTube channel?\xa0\n\nDo you have any tips for people who want to be on YouTube?\xa0\n\nWhat advice would you give to a new YouTuber?\xa0\n\nWhat is the best advice you’ve ever received?\xa0\n\nWhat is the most shocking thing you’ve ever seen on YouTube?\xa0\n\nWhat have you learned from making content on YouTube?\xa0\n\nWhat makes a good YouTube video?\xa0\n\nDo you think YouTube is changing the way we consume content?\xa0\n\nDo you think YouTube is changing the way we think about what’s im

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-1.5-2.1b-base")
model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-1.5-2.1b-base")